In [5]:
import requests
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path
import pandas as pd
import geopandas as gpd
import os
import subprocess

In [2]:
aviris_path_url = 'https://docs.google.com/spreadsheets/d/1g_yWgr4kwGPwVCDiFgCX3tZ9tAkxk-oDXxfqv5U4LeU/export?format=csv' 
kml_dir = r'C:\Users\brunolopez\machineLearning\geospatial\hyperspectral\aviris_kml'
shapefile_dir = r'C:\Users\brunolopez\machineLearning\geospatial\hyperspectral\shapefiles'

In [32]:
class AVIRISDataProcessor:
    def __init__(self, excel_file, kml_dir, shapefile_dir):
        self.excel_file = excel_file
        self.kml_dir = kml_dir
        self.shapefile_dir = shapefile_dir
        self.aviris_path_df = pd.read_excel(self.excel_file)

    def download_aviris_kml(self):
        # Ensure the output directory exists
        Path(self.kml_dir).mkdir(parents=True, exist_ok=True)

        def download_kml(row):
            kml_link = row.link_kml_outline
            if pd.isna(kml_link):
                print(f"Skipping {row.Name} due to missing KML link")
                return  # Skip if the link is NaN
            output_name = f"{row.Name}.kml"
            final_output_path = Path(self.kml_dir) / output_name

            try:
                response = requests.get(kml_link)
                response.raise_for_status()  # Will raise an exception for HTTP errors
                with open(final_output_path, 'wb') as file:
                    file.write(response.content)
                print(f"Downloaded: {output_name}")
            except requests.exceptions.HTTPError as err:
                print(f"HTTP Error for {kml_link}: {err}")
            except requests.exceptions.RequestException as err:
                print(f"Request Exception for {kml_link}: {err}")

        # Using ThreadPoolExecutor to download files in parallel
        with ThreadPoolExecutor(max_workers=5) as executor:
            executor.map(download_kml, self.aviris_path_df.itertuples(index=False))

    def convert_kml_to_shapefiles_gdal(self):
        # Ensure the output directory exists
        Path(self.shapefile_dir).mkdir(parents=True, exist_ok=True)

        # List all KML files in the specified directory
        kml_files = list(Path(self.kml_dir).glob('*.kml'))
        print(f"Found {len(kml_files)} KML files.")

        for kml_file in kml_files:
            print(f"Converting {kml_file} to Shapefile...")
            try:
                shapefile_name = kml_file.stem + '.shp'
                shapefile_path = Path(self.shapefile_dir) / shapefile_name

                # Construct the ogr2ogr command
                cmd = [
                    "ogr2ogr",
                    "-f", "ESRI Shapefile",
                    str(shapefile_path),
                    str(kml_file)
                ]

                # Run the command
                subprocess.run(cmd, check=True)
                print(f"Saved Shapefile: {shapefile_path}")
            except subprocess.CalledProcessError as e:
                print(f"Failed to convert {kml_file}: {e}")

# Usage
excel_file = r'C:\Users\brunolopez\machineLearning\geospatial\hyperspectral\AVIRIS-NG Flight Lines - AVIRIS-NG Flight Lines.xlsx'
kml_dir = r'C:\Users\brunolopez\machineLearning\geospatial\hyperspectral\aviris_kml'
shapefile_dir = r'C:\Users\brunolopez\machineLearning\geospatial\hyperspectral\shapefiles'

processor = AVIRISDataProcessor(excel_file, kml_dir, shapefile_dir)
processor.download_aviris_kml()
processor.convert_kml_to_shapefiles_gdal()


Downloaded: ang20140530t161827.kml
Downloaded: ang20140530t162954.kml
Downloaded: ang20140530t161002.kml
Downloaded: ang20140530t162534.kml
Downloaded: ang20140530t161827.kml
Downloaded: ang20140530t164316.kml
Downloaded: ang20140530t170329.kml
Downloaded: ang20140530t165141.kml
Downloaded: ang20140530t170329.kml
Downloaded: ang20140530t170926.kml
Downloaded: ang20140602t191955.kml
Downloaded: ang20140530t171606.kml
Downloaded: ang20140530t172659.kml
Downloaded: ang20140602t194438.kml
Downloaded: ang20140602t200822.kml
Downloaded: ang20140602t212001.kml
Downloaded: ang20140602t203225.kml
Downloaded: ang20140602t205637.kml
Downloaded: ang20140602t213426.kml
Downloaded: ang20140602t214837.kml
Downloaded: ang20140603t194124.kml
Downloaded: ang20140603t180247.kml
Downloaded: ang20140603t192803.kml
Downloaded: ang20140603t185211.kml
Downloaded: ang20140603t190945.kml
Downloaded: ang20140603t202947.kml
Downloaded: ang20140603t201129.kml
Downloaded: ang20140603t195821.kml
Downloaded: ang20140

Next we begin construction of the 